# Scenario K-Projection Coverage over Kitti Data Sets



In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from nndependability.metrics import ScenarioKProjection

## A. Understanding combinatorial effects in scenario coverage

Load scenario description file, which contains all possible categorizations to partition the input space. 

In [3]:
metric = ScenarioKProjection.Scenario_KProjection_Metric("data/kitti/description.xml")


For the loaded example in data/kitti/description.xml, 
* Condition <ego_vn> (ego velocity to the north) has been discretized to 7 categorizations.
* Condition <left_box_existence> has been discretized to 2 categorizations

One can thus derive the total number of possible scenarios, by picking one item in each condition, equals 7x7x7x7x2x8x9x4x2x8x9x4x2x8x9x4x2x8x9x4, around 2.64x10^14. Such a huge number clearly shows that it is impossible to achieve 100% coverage even when one wants to cover each scenario with one test case. 


Therefore, we would like to have a "relative form" of completeness, where one is able to argue 100% but the confidence of completeness is lower. An analogy apeears in software testing, where compared to path coverage or MC/DC coverage, line coverage is a weaker form - it is easy to achieve "100%" but the "100%" provides weaker guarantee. This brings us to the concept of K-projection coverage.  

## B. Process raw data from Kitti, and compute 2-projection coverage

As the description.xml describes all possible discrete partitionings, the next step is to translate raw data in Kitti into concrete scenarios such that we can compute coverage. We have stored the translated scenarios inside data/kitti/scenarios.xml. 


[Optional] If you want to generate scenarios.xml yourself, you can do it by executing the kitti_scenario_creator.py (available at the root), in the following way

```sh
$ python3 kitti_scenario_creator.py
```

The raw data used in generating scenarios.xml includes
* data/kitti/oxts: GPS and IMU data of the ego vehicle
* tracklet_labels.xml: Object tracking information derived from consecutive images including nearby vehicles and pedestrians. 



In [4]:
metric.addScenariosFromFile("data/kitti/scenarios.xml")


2-projection coverage (without considering domain restrictions): 287/6772


When one adds scenraios into the metric, the current 2-projection coverage is computed (i.e., k=2). One can observe that the computed 2-projection coverage has a denominator of 6772, which is substantially smaller than 2.64x10^14. 

The concept of 2-projection coverage are highly related to the idea of combinatorial testing and coverage arrays, where instead of testing all possible combinations, we just ensure that the set of test cases we have can cover every criterion pair, in contrast to every tuple in 2.64x10^14. For further details, see https://arxiv.org/abs/1805.04333 

## C. Proposing new scenarios

Apart from computing coverage, another task is to generate new scenarios to maximally increase coverage. Here we have some issues where certain combination is just impossible. We use a file to specify the domain restrictions into constraints. 



The domain restriction file in data/kitti/domain-restrictions.xml contains constraints similar to below one.

    <constraint>
        <upperbound>MAX</upperbound>
        <lowerbound>3</lowerbound>
        <item>3, left_box_existence.True</item>
        <item>1, left_box_rotation.0</item>
        <item>1, left_box_type.Misc</item>
        <item>1, left_box_occlusion.0</item>
    </constraint>
    
This constraint says that if  left_box_existence equals False, then 
* left_box_rotation must be 0
* left_box_type must be Misc 
* left_box_occlusion must be 0


[Note] If you see the above computed coverage, it actually says "(without considering domain restrictions)". When one adds domain restrictions, then deriving the denominator equals to the problem of model counting, which is a hard problem itself. 


In [5]:
metric.addDomainRestrictionsFromFile("data/kitti/domain-restrictions.xml")



3.0 <=  + 3.0 C4_0 + 1.0 C5_0 + 1.0 C6_7 + 1.0 C7_0 <= inf
3.0 <=  + 3.0 C8_0 + 1.0 C9_0 + 1.0 C10_7 + 1.0 C11_0 <= inf
3.0 <=  + 3.0 C12_0 + 1.0 C13_0 + 1.0 C14_7 + 1.0 C15_0 <= inf
3.0 <=  + 3.0 C16_0 + 1.0 C17_0 + 1.0 C18_7 + 1.0 C19_0 <= inf


We can ask the solver to propose a scenario candidate which increases coverage while satisfying constraints as specified in domain restriction.

In [6]:
from nndependability.atg.scenario import scenariogen
variableAssignment = scenariogen.proposeScenariocandidate(metric)
metric.writeScenarioToFile(variableAssignment, "local/tmp.xml")

Timeout but feasible solution found in 10 seconds
Maximum possibility for improvement = 6485
Optimal objective value computed from IP = 189

for criterion ego_vn, set it to -3
for criterion ego_ve, set it to -2
for criterion ego_ax, set it to 0
for criterion ego_ay, set it to -3
for criterion left_box_existence, set it to True
for criterion left_box_rotation, set it to 1
for criterion left_box_type, set it to DontCare
for criterion left_box_occlusion, set it to 2
for criterion right_box_existence, set it to True
for criterion right_box_rotation, set it to 0
for criterion right_box_type, set it to Misc
for criterion right_box_occlusion, set it to 3
for criterion closest_box_1_existence, set it to True
for criterion closest_box_1_rotation, set it to 0
for criterion closest_box_1_type, set it to Misc
for criterion closest_box_1_occlusion, set it to 3
for criterion closest_box_2_existence, set it to True
for criterion closest_box_2_rotation, set it to 6
for criterion closest_box_2_type, se

We can now add this new scenario to the existing data set, and one observes that the value of the computed metric has increased.

In [7]:
metric.addScenariosFromFile("local/tmp.xml")

2-projection coverage (without considering domain restrictions): 476/6772


One may continue the process and propose more candidates. 

In [8]:
variableAssignment = scenariogen.proposeScenariocandidate(metric)

Timeout but feasible solution found in 10 seconds
Maximum possibility for improvement = 6296
Optimal objective value computed from IP = 184

for criterion ego_vn, set it to 3
for criterion ego_ve, set it to 2
for criterion ego_ax, set it to -3
for criterion ego_ay, set it to 3
for criterion left_box_existence, set it to True
for criterion left_box_rotation, set it to 6
for criterion left_box_type, set it to Person_sitting
for criterion left_box_occlusion, set it to 3
for criterion right_box_existence, set it to True
for criterion right_box_rotation, set it to 5
for criterion right_box_type, set it to Tram
for criterion right_box_occlusion, set it to 2
for criterion closest_box_1_existence, set it to True
for criterion closest_box_1_rotation, set it to 7
for criterion closest_box_1_type, set it to Van
for criterion closest_box_1_occlusion, set it to 2
for criterion closest_box_2_existence, set it to True
for criterion closest_box_2_rotation, set it to 2
for criterion closest_box_2_type,

In [9]:
metric.writeScenarioToFile(variableAssignment, "local/tmp2.xml")

In [10]:
metric.addScenariosFromFile("local/tmp2.xml")

2-projection coverage (without considering domain restrictions): 660/6772


In [11]:
variableAssignment = scenariogen.proposeScenariocandidate(metric)

Timeout but feasible solution found in 10 seconds
Maximum possibility for improvement = 6112
Optimal objective value computed from IP = 171

for criterion ego_vn, set it to -1
for criterion ego_ve, set it to 1
for criterion ego_ax, set it to -2
for criterion ego_ay, set it to 0
for criterion left_box_existence, set it to True
for criterion left_box_rotation, set it to 3
for criterion left_box_type, set it to Pedestrian
for criterion left_box_occlusion, set it to 1
for criterion right_box_existence, set it to True
for criterion right_box_rotation, set it to 7
for criterion right_box_type, set it to Van
for criterion right_box_occlusion, set it to 0
for criterion closest_box_1_existence, set it to True
for criterion closest_box_1_rotation, set it to 2
for criterion closest_box_1_type, set it to Pedestrian
for criterion closest_box_1_occlusion, set it to 0
for criterion closest_box_2_existence, set it to True
for criterion closest_box_2_rotation, set it to 1
for criterion closest_box_2_ty